# DataFrames

In [1]:
from pyspark.sql import SparkSession

# create a spark session
spark_session = SparkSession.builder.getOrCreate()

# reading a csv file
df = spark_session.read.csv("sample_data/301-people-info.csv", header=True)

# register in the catalog
df.createOrReplaceTempView("person_info")

df.show()

+------------------+--------+--------------------+-----------+-----+-------+-------+
|              name| surname|              street|       city|state|zipcode|sallary|
+------------------+--------+--------------------+-----------+-----+-------+-------+
|              John|     Doe|   120 jefferson st.|  Riverside|   NJ|  08075|  10000|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|  09119|  18900|
|       John Da Man|  Repici|   120 Jefferson St.|  Riverside|   NJ|  08075|  32100|
|           Stephen|   Tyler|7452 Terrace At t...|   SomeTown|   SD|  91234|  32908|
|              null|Blankman|                null|   SomeTown|   SD|  00298|  44221|
|Joan the bone Anne|     Jet|  9th at Terrace plc|Desert City|   CO|  00123|  13900|
|              Jack|McGinnis|        220 hobo Av.|      Phila|   PA|  09119|  18900|
+------------------+--------+--------------------+-----------+-----+-------+-------+



## Queries in a DataFrame

### Select types

```.select(*cols)``` select columns and return a new DataFrame

Show entries in the ```name``` column:

In [2]:
df.select("name").show()

+------------------+
|              name|
+------------------+
|              John|
|              Jack|
|       John Da Man|
|           Stephen|
|              null|
|Joan the bone Anne|
|              Jack|
+------------------+



Show entries in the ```name``` and ```sallary + 1000``` column:

In [3]:
df.select(df["name"],df["sallary"] + 1000).show()

+------------------+----------------+
|              name|(sallary + 1000)|
+------------------+----------------+
|              John|         11000.0|
|              Jack|         19900.0|
|       John Da Man|         33100.0|
|           Stephen|         33908.0|
|              null|         45221.0|
|Joan the bone Anne|         14900.0|
|              Jack|         19900.0|
+------------------+----------------+



Show entries in the ```name```, ```sallary``` and ```sallary > 20000``` column:

In [4]:
df.select("name", "sallary", df['sallary'] > 20000).show()

+------------------+-------+-----------------+
|              name|sallary|(sallary > 20000)|
+------------------+-------+-----------------+
|              John|  10000|            false|
|              Jack|  18900|            false|
|       John Da Man|  32100|             true|
|           Stephen|  32908|             true|
|              null|  44221|             true|
|Joan the bone Anne|  13900|            false|
|              Jack|  18900|            false|
+------------------+-------+-----------------+



In [5]:
from pyspark.sql import Row
from pyspark.sql import functions as f
df1 = spark_session.createDataFrame([Row(a=1, intlist=[1,2,3], mapfield={"a": "b", "c": "d"}),
                                     Row(a=2, intlist=[3,4,5], mapfield={"a": "c", "c": "b"})])

In this example we are using ```.select()``` with ```pyspark.sql.functions.explode()```. This is useful when you have an array or a map in a row.

In [6]:
df1.select("a", f.explode(df1.mapfield).alias("key", "value")).show()

+---+---+-----+
|  a|key|value|
+---+---+-----+
|  1|  a|    b|
|  1|  c|    d|
|  2|  a|    c|
|  2|  c|    b|
+---+---+-----+



In [7]:
df1.select("a", f.explode(df1.intlist).alias("anInt")).show()

+---+-----+
|  a|anInt|
+---+-----+
|  1|    1|
|  1|    2|
|  1|    3|
|  2|    3|
|  2|    4|
|  2|    5|
+---+-----+



### When clause

Evaluate one or a list of conditions. ```.otherwise()``` is None as default.

In [8]:
df2 = df.select("name", f.when(df.state == "PA", df.state).otherwise(None).alias("state"))
df2.show()

+------------------+-----+
|              name|state|
+------------------+-----+
|              John| null|
|              Jack|   PA|
|       John Da Man| null|
|           Stephen| null|
|              null| null|
|Joan the bone Anne| null|
|              Jack|   PA|
+------------------+-----+



```.dropna()``` to drop null values.

In [9]:
df2.dropna(subset="state").show()

+----+-----+
|name|state|
+----+-----+
|Jack|   PA|
|Jack|   PA|
+----+-----+



```.isin()``` checks if the values are in a column

In [10]:
df[df["name"].isin("John Da Man", "Jack")].show()

+-----------+--------+-----------------+---------+-----+-------+-------+
|       name| surname|           street|     city|state|zipcode|sallary|
+-----------+--------+-----------------+---------+-----+-------+-------+
|       Jack|McGinnis|     220 hobo Av.|    Phila|   PA|  09119|  18900|
|John Da Man|  Repici|120 Jefferson St.|Riverside|   NJ|  08075|  32100|
|       Jack|McGinnis|     220 hobo Av.|    Phila|   PA|  09119|  18900|
+-----------+--------+-----------------+---------+-----+-------+-------+



### Like

```.like()``` is a binary operator that verifies if each row in the column contains "McGinnis"

In [11]:
 df.select("name", df["surname"].like("McGinnis")).show()

+------------------+---------------------+
|              name|surname LIKE McGinnis|
+------------------+---------------------+
|              John|                false|
|              Jack|                 true|
|       John Da Man|                false|
|           Stephen|                false|
|              null|                false|
|Joan the bone Anne|                false|
|              Jack|                 true|
+------------------+---------------------+



### Startswith

```.startswith()``` is a binary operator that verifies if each row in the column starts with "S"

In [12]:
df.select("name", df["name"].startswith("S")).show()

+------------------+-------------------+
|              name|startswith(name, S)|
+------------------+-------------------+
|              John|              false|
|              Jack|              false|
|       John Da Man|              false|
|           Stephen|               true|
|              null|               null|
|Joan the bone Anne|              false|
|              Jack|              false|
+------------------+-------------------+



```.endswith()``` is a binary operator that verifies if each row in the column ends with "n"

In [13]:
df.select("name", df["name"].endswith("n")).show()

+------------------+-----------------+
|              name|endswith(name, n)|
+------------------+-----------------+
|              John|             true|
|              Jack|            false|
|       John Da Man|             true|
|           Stephen|             true|
|              null|             null|
|Joan the bone Anne|            false|
|              Jack|            false|
+------------------+-----------------+



```.substr(a, b)``` returns a substring between a and b from each row in a given column.

In [14]:
df.select(df["name"].substr(0,5)).show()

+---------------------+
|substring(name, 0, 5)|
+---------------------+
|                 John|
|                 Jack|
|                John |
|                Steph|
|                 null|
|                Joan |
|                 Jack|
+---------------------+



```.between(a, b)``` returns if the rows in a given column contains values between a and b.

In [21]:
df.select("sallary", df["sallary"].between(30000, 40000)).show()

+-------+-------------------------------------------+
|sallary|((sallary >= 30000) AND (sallary <= 40000))|
+-------+-------------------------------------------+
|  10000|                                      false|
|  18900|                                      false|
|  32100|                                       true|
|  32908|                                       true|
|  44221|                                      false|
|  13900|                                      false|
|  18900|                                      false|
+-------+-------------------------------------------+



In [20]:
df.select("state", df["state"].between("A", "O")).show()

+-----+-------------------------------+
|state|((state >= A) AND (state <= O))|
+-----+-------------------------------+
|   NJ|                           true|
|   PA|                          false|
|   NJ|                           true|
|   SD|                          false|
|   SD|                          false|
|   CO|                           true|
|   PA|                          false|
+-----+-------------------------------+



### groupby

In [26]:
df.groupBy("state").count().show()

+-----+-----+
|state|count|
+-----+-----+
|   NJ|    2|
|   PA|    2|
|   SD|    2|
|   CO|    1|
+-----+-----+



In [23]:
df.filter(df["state"] > "O").show()

+-------+--------+--------------------+--------+-----+-------+-------+
|   name| surname|              street|    city|state|zipcode|sallary|
+-------+--------+--------------------+--------+-----+-------+-------+
|   Jack|McGinnis|        220 hobo Av.|   Phila|   PA|  09119|  18900|
|Stephen|   Tyler|7452 Terrace At t...|SomeTown|   SD|  91234|  32908|
|   null|Blankman|                null|SomeTown|   SD|  00298|  44221|
|   Jack|McGinnis|        220 hobo Av.|   Phila|   PA|  09119|  18900|
+-------+--------+--------------------+--------+-----+-------+-------+

